### Lemmatisierer Evaluierung - qualitativ

In [1]:
import glob
import json
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
%matplotlib inline

In [3]:
FILES = glob.glob("lemmata/*.csv")
corpora = [f.split("baseline-")[1].replace(".csv", "") for f in FILES if "baseline" in f]
data = []
algos = [f.split("lemmata-")[1].split('-')[0] for f in FILES if "lemmata-*-ud-hdt" in f]

for corpus in corpora:
    df = pd.read_csv(f"lemmata/lemmata-baseline-{corpus}.csv", encoding="utf-8", encoding_errors='ignore')
    df.rename(columns = {'lemma_pred':'baseline'}, inplace = True)
    for a in algos[1:]:  # ignore baseline
        try:
            df[a] = pd.read_csv(f"lemmata/lemmata-{a}-{corpus}.csv")["lemma_pred"]
        except Exception as e:  # corpus not lemmatized by algorithm
            print(e, f"{corpus} not lemmatized by {a}\n")
    data.append(df)
    df.to_csv(f"lemmata/{corpus}.csv", encoding="utf-8")

In [4]:
corpora, algos

(['ud-gsd',
  'ud-pud',
  'empirist-cmc-blog-norm',
  'empirist-cmc-blog',
  'empirist-cmc-chat-prof-norm',
  'empirist-cmc-chat-prof',
  'empirist-cmc-chat-social-norm',
  'empirist-cmc-chat-social',
  'empirist-cmc-twitter-norm',
  'empirist-cmc-twitter',
  'empirist-cmc-whatsapp-norm',
  'empirist-cmc-whatsapp',
  'empirist-cmc-wiki-norm',
  'empirist-cmc-wiki',
  'empirist-web-norm',
  'empirist-web',
  'germanc',
  'nosta-d-anselm-norm',
  'nosta-d-anselm-orig',
  'nosta-d-bematac-norm',
  'nosta-d-bematac-orig',
  'nosta-d-falko-norm',
  'nosta-d-falko-orig',
  'nosta-d-kafka-norm',
  'nosta-d-kafka-orig',
  'nosta-d-tuebadz-norm',
  'nosta-d-tuebadz-orig',
  'nosta-d-unicum-norm',
  'nosta-d-unicum-orig',
  'rub2019-novelette',
  'rub2019-sermononline',
  'rub2019-subtitles',
  'rub2019-ted',
  'rub2019-wikipedia',
  'tgermacorp',
  'ud-gsd',
  'ud-hdt',
  'ud-pud'],
 [])